### Agent-Lab: Adaptive RAG Agent

---

In [ ]:
import os
os.chdir('..')

from notebooks import experiment_utils
from app.core.container import Container

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

In [ ]:
# create ollama agent
# agent_id = experiment_utils.create_ollama_agent(agent_type="adaptive_rag")
agent_id = '4c83fbb4-c4b8-475b-9a9d-031da527a5d9'

In [ ]:
# get agent instance
adaptive_rag_agent = container.adaptive_rag_agent()

# Create Graph
workflow_builder = adaptive_rag_agent.get_workflow_builder(agent_id)

In [ ]:
# Nodes definition
# workflow_builder.add_node("retrieve", adaptive_rag_agent.retrieve)
# workflow_builder.add_node("grade_documents", adaptive_rag_agent.grade_documents)
# workflow_builder.add_node("generate", adaptive_rag_agent.generate)
# workflow_builder.add_node("transform_query", adaptive_rag_agent.transform_query)